In [12]:
import sqlite3 as sq
import numpy as np
import pickle
from scipy.optimize import curve_fit
from scipy.stats import linregress
from matplotlib import pyplot as plt
from multiprocessing.pool import Pool
%matplotlib inline
def get_table(conn, table):
    out = None
    with conn:
        out = conn.execute(f"select * from {table};").fetchall();
    out_dict = {k[0]: np.array(k[1:]) for k in out}
    return (get_tau(out), out_dict)

def get_tau(table):
    return np.array(range(1, len(table[0]))) if len(table) >= 1 else None

def plot(data, key, q):
    plt.plot(data[key][0], data[key][1][q])
conn = sq.connect("../crowd.sqlite")

def list_tables(conn, filter_str: str = ""):
    with conn:
        tables = conn.execute("SELECT name FROM main.sqlite_master WHERE type='table'").fetchall();
    return list(filter(lambda t: t.find(filter_str) != -1, map(lambda t: t[0], tables)))

In [15]:
tables = list_tables(conn, "multiDDM")

In [11]:
#Sort into video files

[('video_multiDDM_running8_81',),
 ('video_multiDDM_running8_162',),
 ('video_multiDDM_running8_360',),
 ('video_multiDDM_running8_1024',),
 ('video_multiDDM_running8_405',),
 ('video_multiDDM_running8_216',),
 ('video_multiDDM_running8_729',),
 ('video_multiDDM_running8_288',),
 ('video_multiDDM_running8_640',),
 ('video_multiDDM_running8_108',),
 ('video_multiDDM_running8_243',),
 ('video_multiDDM_running8_486',),
 ('video_multiDDM_running8_135',),
 ('video_multiDDM_running8_810',),
 ('video_multiDDM_running8_540',),
 ('video_multiDDM_running8_64',),
 ('video_multiDDM_crowd14_81',),
 ('video_multiDDM_crowd14_162',),
 ('video_multiDDM_crowd14_360',),
 ('video_multiDDM_crowd14_1024',),
 ('video_multiDDM_crowd14_405',),
 ('video_multiDDM_crowd14_216',),
 ('video_multiDDM_crowd14_729',),
 ('video_multiDDM_crowd14_288',),
 ('video_multiDDM_crowd14_640',),
 ('video_multiDDM_crowd14_108',),
 ('video_multiDDM_crowd14_243',),
 ('video_multiDDM_crowd14_486',),
 ('video_multiDDM_crowd14_135',),